# New baseline with 12?

In [ ]:
from pathlib import Path
import sys
import os
import numpy as np
import matplotlib as mlt

# Had to add this stuff to get notebook to find files, because it is not
# located in the root folder
ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))

from keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import load_img, img_to_array
from keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.utils import image_dataset_from_directory

from bee_tector.config import (
    FULL_DATA_DIR,
    IMAGE_SIZE,
    CURATED_DATA_DIR,
    MODELS_DIR
)
from bee_tector.plots import plot_history
from bee_tector.data import (
    load_datasets,
    undersample_dataset,
    load_selected_classes
)

In [99]:
train_ds, val_ds, test_ds = load_datasets()

Found 3619 files belonging to 12 classes.
Found 781 files belonging to 12 classes.
Found 781 files belonging to 12 classes.


In [100]:
def ensure_rgb(image, label):
    if image.shape[-1] != 3:
        image = tf.image.grayscale_to_rgb(image)
    return image, label

train_ds = train_ds.map(ensure_rgb)
val_ds = val_ds.map(ensure_rgb)
test_ds = test_ds.map(ensure_rgb)

In [109]:
def model_t(shape=(224, 224, 3), num_classes=12):

    data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(0.1, 0.1)
        ], name="data_augmentation")
    # print(shape)
    # loading a pretrainded model tf.keras.applications.EfficientNetB0
    base_model = DenseNet121(
    include_top=False, # Exclude the original ImageNet classification head
    weights='imagenet', # Use pretrained weights from ImageNet
    input_shape=(224, 224, 3))  # Input shape of your images
            
    # this freezes the model to Don't update the weights of the base model during training
    # this way only the newly added layers  will be trained
    base_model.trainable = False

    model = Sequential(name="BeeClassifier")

    model.add(Input(shape=shape))
    model.add(data_augmentation)
    model.add(layers.Rescaling(1./255))  # RESCALE!
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())

    reg = regularizers.l2(1e-5)
    model.add(layers.Dense(128, activation='relu', kernel_regularizer=reg))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))
    

    return model


In [110]:
model = model_t()
model.get_layer('densenet121').trainable = True

In [103]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )

In [104]:
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_ds,
    epochs=1000,
    validation_data=val_ds,
    callbacks=[es],
    verbose=1
)

Epoch 1/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 185s 729ms/step - accuracy: 0.2010 - loss: 2.4134 - val_accuracy: 0.4635 - val_loss: 1.6733
Epoch 2/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 76s 666ms/step - accuracy: 0.5209 - loss: 1.5052 - val_accuracy: 0.5915 - val_loss: 1.2588
Epoch 3/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 77s 676ms/step - accuracy: 0.6180 - loss: 1.1774 - val_accuracy: 0.6159 - val_loss: 1.1923
Epoch 4/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 78s 680ms/step - accuracy: 0.6877 - loss: 0.9598 - val_accuracy: 0.6492 - val_loss: 1.1457
Epoch 5/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 78s 682ms/step - accuracy: 0.7447 - loss: 0.8039 - val_accuracy: 0.6645 - val_loss: 1.0792
Epoch 6/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 78s 681ms/step - accuracy: 0.7646 - loss: 0.7340 - val_accuracy: 0.6530 - val_loss: 1.1137
Epoch 7/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 78s 683ms/step - accuracy: 0.8030 - loss: 0.6340 - val_accuracy: 0.6773 - val_loss: 1.0423
Epoch 8/1000
114/114 ━━━━━━━━━━━━━━━━━━━━ 78s 682ms/step - accuracy:

In [105]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}")

test_loss, test_acc = model.evaluate(test_ds)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 130ms/step - accuracy: 0.7434 - loss: 0.8236
Validation loss: 1.0423, Validation accuracy: 0.6773
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 130ms/step - accuracy: 0.7781 - loss: 0.7104
Test loss: 0.9593, Test accuracy: 0.7068


# 